<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Reseau_RegressionLineaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dans ce carnet nous allons mettre en place une regression linéaire à l'aide d'un réseau de neurones sur notre série temporelle. Pour cela, nous allons suivre les étapes suivantes :
  - Reprendre le code nécessaire pour créer notre série temporelle
  - Définir une fonction permettant de préparer les données d'entrée X et les labels Y pour attaquer notre réseau de régression linéaire
  - Préparer les données X et Y avec la fonction définie précédemment.

In [116]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

# Création de la série temporelle

In [ ]:
# Fonction permettant d'afficher une série temporelle
def affiche_serie(temps, serie, format="-", debut=0, fin=None, label=None):
    plt.plot(temps[debut:fin], serie[debut:fin], format, label=label)
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)

# Fonction permettant de créer une tendance
def tendance(temps, pente=0):
    return pente * temps

# Fonction permettant de créer un motif
def motif_periodique(instants):
    return (np.where(instants < 0.4,                            # Si les instants sont < 0.4
                    np.cos(instants * 2 * np.pi),               # Alors on retourne la fonction cos(2*pi*t)
                    1 / np.exp(3 * instants)))                  # Sinon, on retourne la fonction exp(-3t)

# Fonction permettant de créer une saisonnalité avec un motif
def saisonnalite(temps, periode, amplitude=1, phase=0):
    """Répétition du motif sur la même période"""
    instants = ((temps + phase) % periode) / periode            # Mapping du temps =[0 1 2 ... 1460] => instants = [0.0 ... 1.0]
    return amplitude * motif_periodique(instants)

# Fonction permettant de générer du bruit gaussien N(0,1)
def bruit_blanc(temps, niveau_bruit=1, graine=None):
    rnd = np.random.RandomState(graine)
    return rnd.randn(len(temps)) * niveau_bruit

# Création de la série temporelle
temps = np.arange(4 * 365)                # temps = [0 1 2 .... 4*365] = [0 1 2 .... 1460]
amplitude = 40                            # Amplitude de la la saisonnalité
niveau_bruit = 5                          # Niveau du bruit
offset = 10                               # Offset de la série

serie = offset + tendance(temps, 0.1) + saisonnalite(temps, periode=365, amplitude=amplitude) + bruit_blanc(temps,niveau_bruit)

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie)
plt.title('Série temporelle expérimentale')
plt.show()

Regardons le format de la série temporelle que nous avons pour le moment :

In [ ]:
print(serie.shape)
dataset = tf.data.Dataset.from_tensor_slices(serie)
for vecteur in dataset:
  print(vecteur.numpy())

# Préparation des données X et Y

Définissons la fonction `fenetrage_Dataset` qui va permettre de créer les données X Y pour le réseau de neurones. Cette fonction retourne une classe Dataset et prend comme  paramètres :
 - `serie` : La série temporelle à traiter
 - `taille_fenetre` : La taille de la fenêtre glissante
 - `batch_size` : La nombre de regroupements que l'on souhaite obtenir dans nos données X et Y : par exemple si X=(X1,X2) et Y=(Y1,Y2) alors il faut donner `batch_size = 2`
 - `buffer_melange` : Buffer pour le mélange des données  
   
Cette fonction utilise la méthode [from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) de la classe Dataset de Tensorflow afin d'extraire une coupe depuis la série originale et de créer un dataset qui contient les valeurs de la série :

In [95]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle
# au format X(X1,X2,...Xn) / Y(Y1,Y2,...,Yn)
# X sont les données d'entrées du réseau
# Y sont les labels

def prepare_dataset_XY(serie, taille_fenetre, batch_size, buffer_melange):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre + 1))
  dataset = dataset.shuffle(buffer_melange).map(lambda x: (x[:-1], x[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

Regardons un exemple :

In [ ]:
test = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
dataset = prepare_dataset_XY(serie=test,taille_fenetre=3,batch_size=2,buffer_melange=20)
list(dataset.as_numpy_iterator())

**1. Séparation des données en données pour l'entrainement et la validation**

<img src="https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/Images/Series/illustration1.png?raw=true" width="600">  

In [98]:
temps_separation = 1000

# Extraction des temps et des données d'entrainement
temps_entrainement = temps[:temps_separation]
x_entrainement = serie[:temps_separation]

# Exctraction des temps et des données de valiadation
temps_validation = temps[temps_separation:]
x_validation = serie[temps_separation:]

**2. Préparation des données X et des labels Y**

On commence par créer notre dataset à partir de la série (remarque : les valeurs ci-dessous sont en réalité mélangées) :

<img src="https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/images/split_XY_2.png?raw=true" width="1200"> 

In [111]:
# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 20
batch_size = 32
buffer_melange = 1000

# Création du dataset X,Y
dataset = prepare_dataset_XY(serie,taille_fenetre,batch_size,buffer_melange)

Regardons ce qu'on obtient :

In [ ]:
# Affiche le nombre total d'éléments dans le dataset
print("Nombre total d'éléments dans le dataset : %d" %len(list(dataset.as_numpy_iterator())))

# Affiche le premier élément du dataset
n = 0
print("Premier élément dans le dataset :")
for element in dataset:
  print(element)
  n=n+1
  if (n==1): break


On obtient donc un dataset qui contient :
- 45 éléments de format (32,20) : Ce sont les groupes qui composent les entrées X
- 45 éléments de format (32,) : Ce sont les groupes qui composent les labels Y

**3. Création du réseau de neurone sous Keras**

On met maintenant en place un réseau de neurone pour la régression linéaire constitué :
-  **D'une couche d'entrée avec 1 neurone** qui prend en entrée des **vecteurs 1D de dimension égale à la taille de la fenêtre**.  
On a donc 21 paramètres à calculer (20 poids + 1 offset).  
- On utilise un **taux d'apprentissage de 1e-6**
- On choisit un **optimiseur de type Descente de gradient stochastique et à moment** (MSGD - Momentum Stochastic Gradient Descent) avec un moment fixé à 0.9.  
- On utilise une **fonction d'objectif** de type **erreur moyenne quadratique** (mse - mean squared error)

In [138]:
# Création du modèle de régression linéaire

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(taille_fenetre,)))
model.add(tf.keras.layers.Dense(1))

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 1)                 21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


Une autre possibilité pour construire ce réseau est d'empiler les couches les unes sur les autres comme ci-dessous. Cette solution a l'avantage de pouvoir par la suite accéder directement aux données de la couche qui nous interesse à l'aide du nom de sa variable :

In [141]:
# Construction du réseau avec nomination explicite du nom des couches

couche_0 = tf.keras.layers.Dense(1, input_shape=[taille_fenetre])
model = tf.keras.models.Sequential([couche_0])

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1)                 21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


In [143]:
print(couche_0.input_shape)
print(couche_0.output_shape)

(None, 20)
(None, 1)


**4. Entrainement du réseau**

On entraine le réseau sur 100 périodes. Comme le modèle prend 1 élément (vecteur d'entrée X de dimension 20) à chaque itération, et qu'il y a 45 éléments, il y aura 45 itération par période.

In [146]:
# Lance l'entrainement du modèle

model.fit(dataset,epochs=100,verbose=1)

Epoch 1/100
45/45 [==============================] - 1s 3ms/step - loss: 12560.4597
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 282.5828
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 116.4269
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 94.6391
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 90.5269
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 82.7895
Epoch 7/100
45/45 [==============================] - 0s 3ms/step - loss: 74.9476
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 83.2350
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 81.3899
Epoch 10/100
45/45 [==============================] - 0s 3ms/step - loss: 75.5606
Epoch 11/100
45/45 [==============================] - 0s 3ms/step - loss: 77.6245
Epoch 12/100
45/45 [==============================] - 0s 3ms/step - loss: 70.0998
Epoch 13/100
45/45 [